In [ ]:
from ipyparallel import Client
client = Client(profile='python3')

In [ ]:
from hpc05 import Client
client = Client(profile='pbs', timeout=10)

In [ ]:
dview = client[:]
lview = client.load_balanced_view()
len(dview)

In [ ]:
%%px --local
import sys
sys.path.append('/home/basnijholt/Dropbox/Work/nanowire_current/')
sys.path.append('/home/bnijholt/Dropbox/Work/nanowire_current/')
from collections import namedtuple
import holoviews as hv
from itertools import product
import kwant
import numpy as np
import pandas as pd
from supercurrent import (current_at_phase,
                          discretized_hamiltonian,
                          hoppingkind_at_interface,
                          hoppingkind_in_shape,
                          make_params,
                          peierls,
                          SimpleNamespace,
                          I_c)
import sympy

In [ ]:
%matplotlib inline
hv.notebook_extension()
import sympy.interactive
sympy.interactive.init_printing('mathjax')


# 3D test system

In [ ]:
from supercurrent import k_B
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz_
onsite_lead = lambda site, p: -p.mu * sz_ + p.delta * sx_
hops = lambda site1, site2, p: -p.t * sz_

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3))
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    range_Z = range(Z)
    syst[(lat(x, y, z) for x in range_X for y in range_Y for z in range_Z)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range_Y for z in range_Z)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping



delta = 0.01
T = 0.5 * delta / k_B
syst, hopping = make_test_system(X=3, Y=3, Z=3)
p = SimpleNamespace(mu=0., t=1., delta=1)

phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=None, max_frequencies=200) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# 3D system with Discretizer Hamiltonian, squared system

In [ ]:
def make_test_system(X, Y, Z, a=10):
    tb_normal, tb_sc = discretized_hamiltonian(a)[:-1]
    lat = tb_normal.lattice
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = tb_normal.onsite

    for hop, val in tb_normal.hoppings.items():
        syst[hop] = val

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = tb_sc.onsite

    for hop, val in tb_sc.hoppings.items():
        lead[hop] = val

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

T = 60e-3

syst, hopping = make_test_system(X=2, Y=3, Z=3)
p = make_params(Delta=0.25, alpha=20, mu=15)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=None, max_frequencies=200) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# Simpler way (but slightly slower)

In [ ]:
from supercurrent import matsubara_frequency, k_B
def null_G(syst, p, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return gf.data[::2, ::2]

def current_from_G_0(T, G_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

matsfreqs = 200
G_0s = [null_G(syst, p, T, n) for n in range(matsfreqs)]
H12 = hopping(syst, args=[p])
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(T, G_0s, H12, phase) for phase in phases]
I = np.array(I)
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# Test 2D system

In [ ]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz_
onsite_lead = lambda site, p: -p.mu * sz_ + p.delta * sx_
hops = lambda site1, site2, p: -p.t * sz_

def make_test_system_2D(X=3, Y=2):
    lat = kwant.lattice.square()
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    syst[(lat(x, y) for x in range_X for y in range_Y)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y) for x, y in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y) for x, y in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0)))
    lead[(lat(0, y) for y in range_Y)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


In [ ]:
syst, _ = make_test_system_2D()
kwant.plot(syst)

# 1D


In [ ]:
%%px --local
from discretizer import Discretizer, momentum_operators
sx, sy, sz = [sympy.physics.matrices.msigma(i) for i in range(1, 4)]
s0 = sympy.eye(2)
from sympy.physics.quantum import TensorProduct as kr

def discretized_hamiltonian(a, holes=True):
    """Discretizes a Hamiltonian.
    Parameters:
    -----------
    a : int
        Lattice constant in nm.
    holes : bool
        If False, Hamiltonian will only be in spin-space,
        if True also in particle-hole space (BdG Hamiltonian),
        used for calculating Majorana decay length.
    """
    k_x, *_ = momentum_operators
    t, B_x, B_y, B_z, mu_B, mu, mu_sm, mu_sc, alpha, g, V, Delta = sympy.symbols(
        't B_x B_y B_z mu_B mu mu_sm mu_sc alpha g V Delta', real=True)

    if not holes:
        ham = ((t * k_x**2 - mu) * s0 + alpha * (- k_x * sy) +
               0.5 * g * mu_B * (B_x * sx + B_y * sy + B_z * sz))
    else:
        ham = ((t * k_x**2 - mu) * kr(s0, sz) +
                   alpha * (- k_x * kr(sy, sz)) +
                   0.5 * g * mu_B * (B_x * kr(sx, s0) + B_y * kr(sy, s0)) +
                   Delta * kr(s0, sx))
    
    args = dict(lattice_constant=a, discrete_coordinates=set(['x']))
    tb_sm = Discretizer(ham.subs(mu, mu_sm).subs(Delta, 0), **args)
    tb_sc = Discretizer(ham.subs([(g, 0), (mu, mu_sc), (alpha, 0)]), **args)
    return tb_sm, tb_sc


def make_test_system_1D(X, L=40, a=10):
    tb_normal, tb_sc = discretized_hamiltonian(a)
    lat = tb_normal.lattice
    syst = kwant.Builder()
    syst[(lat(x) for x in range(X))] = tb_normal.onsite
    syst[(lat(x) for x in range(-L, 0))] = tb_sc.onsite
    syst[(lat(x) for x in range(X, X+L))] = tb_sc.onsite
    
    for hop, val in tb_normal.hoppings.items():
        syst[hop] = val

    l_cut = [lat(x) for x in np.squeeze([i.tag for i in syst.sites()]) if x == X//2]
    r_cut = [lat(x) for x in np.squeeze([i.tag for i in syst.sites()]) if x == X//2 + 1]
    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a,)))
    lead[lat(0)] = tb_sc.onsite
    
    for hop, val in tb_sc.hoppings.items():
        lead[hop] = val

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

syst, hopping = make_test_system_1D(50, L=1)
p = make_params(Delta=0.25, alpha=20, mu_sm=1, mu_sc=1)

In [ ]:
# vals = named_product(alpha=np.linspace(0, 30, 60), B_x=np.linspace(0, 1.5, 50), mu=[0.1, 0.3 ,1, 3, 10, 30])
# current_phase = lview.map_async(
#     lambda x: I_c(syst, hopping, p.update(alpha=x.alpha, B_x=x.B_x, mu_sc=x.mu, mu_sm=x.mu), T=1), vals)

# current_phase.wait_interactive()

# current_phase = current_phase.result()
# T = 1
# vals = namespace_product(alpha=np.linspace(0, 25, 50), B_x=np.linspace(0, 1, 50), mu=[0, 0.1, 0.3 ,1, 3, 10, 30])[:100]
# current_phase = [I_c(syst, hopping, p.update(alpha=x.alpha, B_x=x.B_x, mu_sc=x.mu, mu_sm=x.mu), T) for x in vals]
# df = pd.concat((pd.DataFrame(vals), pd.DataFrame(current_phase)), axis=-1)

In [ ]:
alphas = np.linspace(0, 100, 200)
B_xs = np.linspace(0, 1.5, 200)
mus = [0.1, 0.3 ,1, 3, 10, 30]
vals = list(product(alphas, B_xs, mus))
current_phase = lview.map_async(
    lambda x: I_c(syst, hopping, p.update(alpha=x[0], B_x=x[1], mu_sc=x[2], mu_sm=x[2]), T=1), vals)

current_phase.wait_interactive()

current_phase = current_phase.result()

In [ ]:
df = pd.concat((pd.DataFrame(vals, columns=['alpha', 'B_x', 'mu']), pd.DataFrame(current_phase)), axis=1)
# df.to_hdf('1d_alpha_vs_B_x.hdf', 'all_data', mode='w')

In [ ]:
def Image(group, x, y, z):
    bounds = (df[x].min(), df[y].min(), df[x].max(), df[y].max())
    table = group.pivot_table(index=[x], columns=[y], values=[z])
    return hv.Image(np.rot90(table), kdims=[x, y], vdims=[z], bounds=bounds)

In [ ]:
%%opts Image (cmap='viridis') Layout {+axiswise}
ims = [(mu, Image(group, 'B_x', 'alpha', 'current_c')) for mu, group in df.groupby('mu')]
hm = hv.HoloMap(ims, kdims=['mu'])
hv.Layout([im.relabel('mu = {} meV'.format(mu)) for mu, im in ims]).cols(3)

In [ ]:
syst, hopping = make_test_system_1D(50, L=1)
p = make_params(Delta=0.25, alpha=25, mu=3)
p.mu_sc = p.mu_sm = p.mu
I = np.array([I_c(syst, hopping, p, T=1)['current_c'] for p.B_x in B_xs])
hv.Curve((B_xs, I), kdims=['B_x'], vdims=['$I_c$'])

In [ ]:
syst, hopping = make_test_system_1D(50, L=1)
p = make_params(Delta=0.25, alpha=20, mu=30)
p.mu_sc = p.mu_sm = p.mu
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([current_at_phase(syst, hopping, p, T=1, H_0_cache, phase, tol=1e-2, max_frequencies=500) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

In [ ]:
%%opts Path [aspect='square']
bands = kwant.physics.Bands(syst.leads[1], args=[p])
ks = np.linspace(-3, 3, 300)
Es = np.array([bands(k=k) for k in ks])
hv.Path((ks, Es))[:, -1:1]

In [ ]:
import scipy.sparse.linalg as sla
from scipy.sparse import identity
import kwant.linalg.mumps as mumps

def sparse_diag(matrix, k, sigma, **kwargs):
    """Call sla.eigsh with mumps support.
    Please see scipy.sparse.linalg.eigsh for documentation.
    """
    class LuInv(sla.LinearOperator):
        def __init__(self, A):
            inst = mumps.MUMPSContext()
            inst.analyze(A, ordering='pord')
            inst.factor(A)
            self.solve = inst.solve
            sla.LinearOperator.__init__(self, A.dtype, A.shape)
        def _matvec(self, x):
            return self.solve(x.astype(self.dtype))
    opinv = LuInv(matrix - sigma * identity(matrix.shape[0]))
    return sla.eigsh(matrix, k, sigma=sigma, OPinv=opinv, **kwargs)

def spectrum(syst, p):
    ham = syst.hamiltonian_submatrix(args=[p], sparse=True)
    eigenvals, eigenvecs = sparse_diag(ham, 20, 0)
    return np.sort(eigenvals)

In [ ]:
syst, hopping = make_test_system_1D(X=64, L=40)
p = make_params(Delta=0.25, alpha=20, mu_sc=1, mu_sm=1)
alphas = np.linspace(0, 50, 100)
p.B_x = 0.5
Es = np.array([spectrum(syst, p) for p.alpha in alphas])

In [ ]:
%%output size=200
%%opts Path [aspect='square'] HLine (color='k')
hv.Path((alphas, np.sort(Es)),  kdims=[r'$\alpha$', 'E'])[:, -p.Delta:p.Delta]